# 📊 Análisis de Ventas - Informe Completo

Este notebook contiene un análisis completo de los datos de ventas de la tienda, incluyendo:

- Carga y exploración de datos
- Cálculo de ventas totales por mes
- Identificación del producto más vendido y con mayores ingresos
- Visualizaciones interactivas
- Conclusiones y recomendaciones

---


In [ ]:
# Importar librerías necesarias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Configurar estilo de gráficos
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

print("Librerías importadas correctamente ✓")


## 1. Carga de Datos

Primero, cargamos el archivo CSV con los datos de ventas.


In [ ]:
# Cargar datos del CSV
archivo_csv = 'ventas.csv'
df = pd.read_csv(archivo_csv)

# Convertir fecha a datetime
df['fecha'] = pd.to_datetime(df['fecha'])

# Calcular ingresos por venta (cantidad * precio)
df['ingresos'] = df['cantidad'] * df['precio']

print(f"✓ Datos cargados: {len(df)} registros")
print(f"✓ Periodo: {df['fecha'].min().strftime('%d/%m/%Y')} - {df['fecha'].max().strftime('%d/%m/%Y')}")
print(f"✓ Productos: {df['producto'].nunique()} diferentes")

# Mostrar primeras filas
df.head()


### Información del Dataset

Veamos información general sobre los datos cargados.


In [ ]:
# Información del dataset
print("Información del DataFrame:")
print(df.info())
print("\n" + "="*60)
print("Estadísticas descriptivas:")
print("="*60)
df.describe()


## 2. Ventas Totales por Mes

Agrupamos las ventas por mes y calculamos los totales de ingresos y cantidad vendida.


In [ ]:
# Extraer año-mes
df['año_mes'] = df['fecha'].dt.to_period('M')

# Agrupar por mes y calcular totales
ventas_por_mes = df.groupby('año_mes').agg({
    'ingresos': 'sum',
    'cantidad': 'sum'
}).reset_index()

ventas_por_mes.columns = ['mes', 'ingresos_totales', 'cantidad_total']
ventas_por_mes['mes_str'] = ventas_por_mes['mes'].astype(str)

print("Ventas totales por mes:")
print("="*60)
ventas_por_mes[['mes_str', 'ingresos_totales', 'cantidad_total']]


### Visualización: Ventas por Mes


In [ ]:
# Crear figura con dos subplots
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico 1: Ventas por mes (ingresos)
ax1 = axes[0]
ax1.plot(ventas_por_mes['mes_str'], ventas_por_mes['ingresos_totales'], 
         marker='o', linewidth=2, markersize=8, color='#2ecc71')
ax1.set_title('Ventas Totales por Mes (Ingresos)', fontsize=14, fontweight='bold', pad=15)
ax1.set_xlabel('Mes', fontsize=12)
ax1.set_ylabel('Ingresos ($)', fontsize=12)
ax1.grid(True, alpha=0.3, linestyle='--')
ax1.tick_params(axis='x', rotation=45)

# Agregar valores en los puntos
for i, (mes, ingreso) in enumerate(zip(ventas_por_mes['mes_str'], ventas_por_mes['ingresos_totales'])):
    ax1.annotate(f'${ingreso:.0f}', 
                (i, ingreso), 
                textcoords="offset points", 
                xytext=(0,10), 
                ha='center',
                fontsize=9)

# Gráfico 2: Ventas por mes (cantidad)
ax2 = axes[1]
ax2.bar(ventas_por_mes['mes_str'], ventas_por_mes['cantidad_total'], 
        color='#3498db', alpha=0.8, edgecolor='black', linewidth=1.5)
ax2.set_title('Cantidad de Unidades Vendidas por Mes', fontsize=14, fontweight='bold', pad=15)
ax2.set_xlabel('Mes', fontsize=12)
ax2.set_ylabel('Cantidad Total', fontsize=12)
ax2.grid(True, alpha=0.3, linestyle='--', axis='y')
ax2.tick_params(axis='x', rotation=45)

# Agregar valores en las barras
for i, (mes, cantidad) in enumerate(zip(ventas_por_mes['mes_str'], ventas_por_mes['cantidad_total'])):
    ax2.text(i, cantidad, f'{cantidad:.0f}', 
            ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()


In [ ]:
# Agrupar por producto
ventas_por_producto = df.groupby('producto').agg({
    'cantidad': 'sum',
    'ingresos': 'sum',
    'precio': 'mean'
}).reset_index()

ventas_por_producto.columns = ['producto', 'cantidad_total', 'ingresos_totales', 'precio_promedio']
ventas_por_producto = ventas_por_producto.sort_values('ingresos_totales', ascending=False)

print("Resumen por producto:")
print("="*60)
ventas_por_producto


### Producto Más Vendido y con Mayores Ingresos


In [ ]:
# Producto más vendido (en cantidad)
producto_mas_vendido = ventas_por_producto.sort_values('cantidad_total', ascending=False).iloc[0]

# Producto con mayores ingresos
producto_mayor_ingresos = ventas_por_producto.iloc[0]

print("🏆 PRODUCTO MÁS VENDIDO (cantidad)")
print("="*60)
print(f"   Producto: {producto_mas_vendido['producto']}")
print(f"   Cantidad total: {producto_mas_vendido['cantidad_total']:.0f} unidades")
print(f"   Ingresos: ${producto_mas_vendido['ingresos_totales']:.2f}")

print("\n💰 PRODUCTO CON MAYORES INGRESOS")
print("="*60)
print(f"   Producto: {producto_mayor_ingresos['producto']}")
print(f"   Ingresos totales: ${producto_mayor_ingresos['ingresos_totales']:.2f}")
print(f"   Cantidad vendida: {producto_mayor_ingresos['cantidad_total']:.0f} unidades")


### Visualización: Top 5 Productos


In [ ]:
# Top 5 productos por ingresos
top5_productos = ventas_por_producto.head(5)

# Crear figura con dos subplots
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Gráfico 1: Top 5 productos por ingresos (barras horizontales)
ax3 = axes[0]
top5_ordenado = top5_productos.sort_values('ingresos_totales', ascending=True)
colores = sns.color_palette("viridis", len(top5_ordenado))
ax3.barh(top5_ordenado['producto'], top5_ordenado['ingresos_totales'], 
         color=colores, alpha=0.8, edgecolor='black', linewidth=1.5)
ax3.set_title('Top 5 Productos por Ingresos', fontsize=14, fontweight='bold', pad=15)
ax3.set_xlabel('Ingresos Totales ($)', fontsize=12)
ax3.set_ylabel('Producto', fontsize=12)
ax3.grid(True, alpha=0.3, linestyle='--', axis='x')

# Agregar valores en las barras
for i, (producto, ingreso) in enumerate(zip(top5_ordenado['producto'], top5_ordenado['ingresos_totales'])):
    ax3.text(ingreso, i, f'${ingreso:.0f}', 
            ha='left', va='center', fontsize=10, fontweight='bold', 
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7))

# Gráfico 2: Top 5 productos por cantidad vendida
ax4 = axes[1]
top5_cantidad = ventas_por_producto.sort_values('cantidad_total', ascending=False).head(5)
top5_cantidad_ordenado = top5_cantidad.sort_values('cantidad_total', ascending=True)
colores2 = sns.color_palette("plasma", len(top5_cantidad_ordenado))
ax4.barh(top5_cantidad_ordenado['producto'], top5_cantidad_ordenado['cantidad_total'], 
         color=colores2, alpha=0.8, edgecolor='black', linewidth=1.5)
ax4.set_title('Top 5 Productos por Cantidad Vendida', fontsize=14, fontweight='bold', pad=15)
ax4.set_xlabel('Cantidad Total (unidades)', fontsize=12)
ax4.set_ylabel('Producto', fontsize=12)
ax4.grid(True, alpha=0.3, linestyle='--', axis='x')

# Agregar valores en las barras
for i, (producto, cantidad) in enumerate(zip(top5_cantidad_ordenado['producto'], top5_cantidad_ordenado['cantidad_total'])):
    ax4.text(cantidad, i, f'{cantidad:.0f}', 
            ha='left', va='center', fontsize=10, fontweight='bold',
            bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.7))

plt.tight_layout()
plt.show()


## 4. Resumen Final y Conclusiones

### Métricas Generales


In [ ]:
# Calcular métricas finales
total_ingresos = df['ingresos'].sum()
total_unidades = df['cantidad'].sum()
promedio_venta = df['ingresos'].mean()
num_productos = df['producto'].nunique()

mes_max_ingresos = ventas_por_mes.loc[ventas_por_mes['ingresos_totales'].idxmax()]
mes_max_cantidad = ventas_por_mes.loc[ventas_por_mes['cantidad_total'].idxmax()]

print("="*60)
print("RESUMEN FINAL")
print("="*60)
print(f"Total de ingresos: ${total_ingresos:,.2f}")
print(f"Total de unidades vendidas: {total_unidades:,}")
print(f"Promedio de ingresos por venta: ${promedio_venta:.2f}")
print(f"Número de productos diferentes: {num_productos}")
print(f"Periodo analizado: {df['fecha'].min().strftime('%d/%m/%Y')} - {df['fecha'].max().strftime('%d/%m/%Y')}")
print("\n" + "="*60)
print("MEJORES RESULTADOS")
print("="*60)
print(f"Mejor mes en ingresos: {mes_max_ingresos['mes_str']} (${mes_max_ingresos['ingresos_totales']:,.2f})")
print(f"Mejor mes en cantidad: {mes_max_cantidad['mes_str']} ({mes_max_cantidad['cantidad_total']:.0f} unidades)")
print(f"Producto más vendido: {producto_mas_vendido['producto']} ({producto_mas_vendido['cantidad_total']:.0f} unidades)")
print(f"Producto con mayores ingresos: {producto_mayor_ingresos['producto']} (${producto_mayor_ingresos['ingresos_totales']:,.2f})")


### Conclusiones y Recomendaciones

**Hallazgos Principales:**

1. **Mejor mes en ingresos**: El mes con mayores ingresos fue **{mes_max_ingresos['mes_str']}** con ${mes_max_ingresos['ingresos_totales']:,.2f}
2. **Mejor mes en cantidad**: El mes con más unidades vendidas fue **{mes_max_cantidad['mes_str']}** con {mes_max_cantidad['cantidad_total']:.0f} unidades
3. **Producto estrella**: El producto **{producto_mayor_ingresos['producto']}** genera los mayores ingresos, mientras que el producto **{producto_mas_vendido['producto']}** es el más vendido en cantidad
4. **Tendencia**: {'Crecimiento positivo' if ventas_por_mes.iloc[-1]['ingresos_totales'] > ventas_por_mes.iloc[0]['ingresos_totales'] else 'Estable'} en los ingresos a lo largo del periodo analizado

**Recomendaciones:**

- ✅ Incrementar el stock y promoción del producto **{producto_mayor_ingresos['producto']}** debido a su alto rendimiento en ingresos
- ✅ Analizar estrategias para aumentar las ventas del producto **{producto_mas_vendido['producto']}** que tiene alta rotación
- ✅ Replicar las estrategias exitosas del mes **{mes_max_ingresos['mes_str']}** en otros periodos
- ✅ Considerar promociones para productos con menor rendimiento para mejorar su participación en el mercado

---

*Notebook generado el {datetime.now().strftime('%d de %B de %Y')}*
